# Deciding What Belongs in a Class
This chapter focus on how to decide what belongs in class.

Firstly, we need to group methods into class. However, we cannot do this correctly at early stage of project because our applications need to be changed in the future.

Therefore, we have to organize code to be easy to change. To achieve this, the code should follow TRUE principle;

* Transparent: The consequences of change should be obvious in the code that is changing and in distant code relies upon it
* Reasonable: The cost of any change should be proportional to the benefits the change achieves
* Usable: Existing code should be usable in new and unexpected contexts
* Exemplary: The code itself should encourage those who change it to perpetuate these qualities

TRUE principle ensures each class has a single responsibility.

# Creating Classes That Have a Single Responsibility


Code sample is about bicycle and its gear. Gear has chainring and cog, and we can calculate gear ratio dividing chainring by cog of gear.

In [ ]:
chainring = 52
cog = 11
ratio = chainring / float(cog)
print(ratio)

chainring = 30
cog = 27
ratio = chainring / float(cog)
print(ratio)

4.7272727272727275
1.1111111111111112


When you write something into code, nouns are candidates to be classes, in this case bicycle and gear. Bicycle can be class but it doesn’t have data or behaviors in this example. On the other hand, gear has, such as chainrings, cogs and ratios, so we should create Gear class.

In [ ]:
class Gear:
  def __init__(self, chainring, cog):
    self.chainring = chainring
    self.cog = cog

  def ratio(self):
    return self.chainring / float(self.cog)

print(Gear(52, 11).ratio())
print(Gear(30, 27).ratio())

4.7272727272727275
1.1111111111111112


We calculated gear ratio, but cyclists also want to know gear inches. The formula follows:

* `gear inches = wheel diameter * gear ratio`
* `wheel diameter = rim diameter + twice tire diameter`
We can easily add this new behavior:

In [ ]:
class Gear:
  def __init__(self, chainring, cog, rim, tire):
    self.chainring = chainring
    self.cog = cog
    self.rim = rim
    self.tire = tire

  def ratio(self):
    return self.chainring / float(self.cog)

  def gear_inches(self):
    return ratio * (self.rim + (self.tire * 2))

print(Gear(52, 11, 26, 1.5).gear_inches())
print(Gear(52, 11, 24, 1.25).gear_inches())

32.22222222222222
29.444444444444446


But we just want to know ratio as before, the code causes bug.

In [ ]:
print(Gear(52, 11).ratio()) #TypeError: __init__() missing 2 required positional arguments: 'rim' and 'tire'

TypeError: ignored

Based on these, is this the best way to organize the code? If you know this application will not change forever, this is good enough. However, most applications will change and you also don’t know what changes will come.

# Writing Code That Embraces Change
Here are a few well-known techniques that you can use to create code that embraces change.

## Depend on Behavior, Not Data


### Hide Instance Variables



```
def cog(self):
  return self.cog
```
Implementing this method changes cog from data (which is referenced all over) to behavior (which is defined once). If the cog is referred to 10 times and suddenly change its specification, you don’t need to change code 10 places.


In [ ]:
class Gear:
  def __init__(self, chainring, cog, rim, tire):
    self.chainring = chainring
    self.cog = cog
    self.rim = rim
    self.tire = tire

  def cog(self):
    return self.cog

  def ratio(self):
    return self.chainring / float(cog)

  def gear_inches(self):
    return self.ratio() * (self.rim + (self.tire * 2))

print(Gear(52, 11, 26, 1.5).gear_inches()) #32.22222222222222
print(Gear(52, 11, 24, 1.25).gear_inches()) #29.444444444444446

32.22222222222222
29.444444444444446


### Hide Data Structures



In [ ]:
class ObscuringReference:
  def __init__(self, data):
    self.data = data

  def diameters(self):
    return [cell[0] + (cell[1] * 2) for cell in self.data]

data = [
  [622, 20],
  [622, 23],
  [559, 30],
  [559, 40]
]
print(ObscuringReference(data).diameters())

[662, 668, 619, 639]


This example is not bad, but to do anything useful, each sender of data must have complete knowledge of what piece of data is at which index in the array.

In this case, sender has to know that rims are at [0] and tires are at [1]. If data structure changes, then this code must change. The knowledge that rims are at [0] should not be duplicated in your application because it’s not DRY (Don’t Repeat Yourself).

In [ ]:
from collections import namedtuple

class RevealingReference:
  Wheel = namedtuple("Wheel", ("rim", "tire"))
  
  def __init__(self, data):
    self.wheels = self.wheelify(data)

  def diameters(self):
    return [wheel.rim + (wheel.tire * 2) for wheel in self.wheels]
  
  def wheelify(self, data):
    return [RevealingReference.Wheel(cell[0], cell[1]) for cell in data]

data = [
  [622, 20],
  [622, 23],
  [559, 30],
  [559, 40]
]
print(RevealingReference(data).diameters())

[662, 668, 619, 639]


Implementing as above, diameters method now has no knowledge of the internal structure of the array. And all knowledge of the structure has been isolated inside the wheelify method, so we don’t need to change code if the structure changes, for example insert into new data between [0] and [1].

## Enforce Single Responsibility Everywhere


### Extract Extra Responsibility from Methods

This method, diameters method of RevealingReference class, has two responsibilities:

* Iterates over the wheels
* Calculates the diameter of each wheel
```
def diameters(self):
  return [wheel.rim + (wheel.tire * 2) for wheel in self.wheels]
```

We can simplify the code by separating it into two methods, each has single responsibility.


```
#Iterates over the wheels
def diameters(self):
  return [diameters(wheel) for wheel in self.wheels]
#Calculates the diameter of each wheel
def diameter(self, wheel):
  return wheel.rim + (wheel.tire * 2)
```

In addition, we recall gear_inches method of Gear class. This also has two responsibilities:
* Iterates over the wheels
* Calculates the diameter of each wheel


```
def gear_inches(self):
  return ratio * (self.rim + (self.tire * 2))
```



We can simplify like this:

```
def gear_inches(self):
  return ratio * diameter
def diameter(self):
  return self.rim + (self.tire * 2)
```

Why do we need to take care of single responsibility? That has following benefits:
* Expose previously hidden qualities
* Avoid the need for comments
* Encourage reuse
* Are easy to move to another class

### Isolate Extra Responsibilities in Classes



Once you simplify each method that has a single responsibility, now you see the class responsibility.

We calculated diameter in gear_inches method of Gear class, but diameter is more likely behavior of Wheel. Therefore, we can create Wheel class that contains diameter method.

In [ ]:
class Wheel:
  def __init__(self, rim, tire):
    self.rim = rim
    self.tire = tire
  def diameter(self):
    return self.rim + (self.tire * 2)
print(Wheel(26, 1.5).diameter())

29.0


Then, now we can calculate diameter in Gear class, through Wheel class.

# Final code

In [ ]:
class Gear:
  def __init__(self, chainring, cog, wheel=None):
    self.chainring = chainring
    self.cog = cog
    self.wheel = wheel
  def ratio(self):
    return self.chainring / float(self.cog)
  def gear_inches(self):
    return self.ratio() * self.wheel.diameter()
class Wheel:
  def __init__(self, rim, tire):
    self.rim = rim
    self.tire = tire
  def diameter(self):
    return self.rim + (self.tire * 2)
wheel = Wheel(26, 1.5)
print(Gear(52, 11, wheel).gear_inches())
print(Gear(52, 11).ratio())